In [19]:
import sys
sys.path.append('/home/abk171/ultralytics')
from ultralytics import YOLO
import SimpleITK as sitk
from PIL import Image
import random
import os
import pydicom
import numpy as np
from tqdm import tqdm
import wandb

In [11]:
restructured_root = 'data_yolo'
image_dir = os.path.join(restructured_root, 'images')
image_train_dir = os.path.join(image_dir, 'train')
image_val_dir = os.path.join(image_dir, 'val')
image_test_dir = os.path.join(image_dir, 'test')
label_dir = os.path.join(restructured_root, 'labels')
label_train_dir = os.path.join(label_dir, 'train')
label_val_dir = os.path.join(label_dir, 'val')

## make these directories
os.makedirs(restructured_root, exist_ok=True)
os.makedirs(image_dir, exist_ok=True)
os.makedirs(image_train_dir, exist_ok=True)
os.makedirs(image_val_dir, exist_ok=True)
os.makedirs(image_test_dir, exist_ok=True)
os.makedirs(label_dir, exist_ok=True)
os.makedirs(label_train_dir, exist_ok=True)
os.makedirs(label_val_dir, exist_ok=True)

In [6]:
anno_dir = '/data_vault/hexai02/CarpalTunnel/Annotations'
dicom_dir = '/data_vault/hexai02/CarpalTunnel/Images'

In [7]:
def load_mask(mask_path):
    mask = sitk.ReadImage(mask_path)
    return sitk.GetArrayFromImage(mask)[0][: 450, 200: 1300]

def get_bounding_box(mask, padding=10):
    """
    Compute the bounding box around the nonzero mask region with optional padding.
    
    Args:
        mask (numpy array): Binary mask where nonzero pixels represent the object.
        padding (int): Extra pixels to add around the bounding box.
    
    Returns:
        (y_min, y_max, x_min, x_max): Cropped bounding box coordinates.
    """

    coords = np.argwhere(mask > 0)  

    if coords.size == 0:
        return None  # No object found

    # Get bounding box coordinates
    y_min, x_min = coords.min(axis=0)
    y_max, x_max = coords.max(axis=0)


    y_min = max(y_min - padding, 0)
    x_min = max(x_min - padding, 0)
    y_max = min(y_max + padding, mask.shape[0])
    x_max = min(x_max + padding, mask.shape[1])

    return y_min, y_max, x_min, x_max

def load_dicom(dicom_path):
    dicom_data = pydicom.dcmread(dicom_path)
    return dicom_data.pixel_array[: 450, 200: 1300,0]

def dicom_to_jpg(image):
    image = ((image - image.min()) / (image.max() - image.min()) * 255).astype(np.uint8)
    img = Image.fromarray(image)  # Convert to PIL Image
    return img.convert('L')

def anno_to_yolo(anno_file, height, width):
    mask = load_mask(anno_file)
    ymin, ymax, xmin, xmax = get_bounding_box(mask, padding=10)
    
    center_x = (xmin + xmax) / (2 * width)
    center_y = (ymin + ymax) / (2 * height)
    width = (xmax - xmin) / width
    height = (ymax - ymin) / height

    return [center_x, center_y, width, height]

In [12]:
dicom_paths = sorted(os.listdir(dicom_dir))

badset = {'97.dcm', '95.dcm', '178.dcm'}
for i in tqdm(range(len(dicom_paths)), desc="Processing DICOMs"):
    if dicom_paths[i] in badset:
        continue
    anno_path = dicom_paths[i].split('.')[0] + '.nii.gz'
    dicom_path = os.path.join(dicom_dir, dicom_paths[i])
    anno_path = os.path.join(anno_dir, anno_path)
    
    img_array = load_dicom(dicom_path)
    img_jpg = dicom_to_jpg(img_array)
    # height, width, dim = img_array.shape
    height, width = img_array.shape
    bbox = anno_to_yolo(anno_path, height, width)


    img_filename = dicom_paths[i].replace('.dcm', '.jpg')
    label_filename = img_filename.replace('.jpg', '.txt')
    
    if random.random() < 0.8:
        img_jpg.save(os.path.join(image_train_dir, img_filename))
        with open(os.path.join(label_train_dir, label_filename), 'w') as f:
            f.write(f"0 {' '.join(map(str, bbox))}\n")

    else:
        img_jpg.save(os.path.join(image_val_dir, img_filename))
        with open(os.path.join(label_val_dir, label_filename), 'w') as f:
            f.write(f"0 {' '.join(map(str, bbox))}\n")

    

Processing DICOMs: 100%|████████████████████████████████████████████| 123/123 [00:09<00:00, 13.03it/s]


In [8]:
dicom_paths[i]

'96.dcm'

In [13]:
%%writefile data_yolo.yaml
path: /home/abk171/hexai_work/data_yolo
train: images/train
val: images/val
test: images/test

names:
    0: ulnar

Overwriting data_yolo.yaml


In [18]:
model = YOLO('yolo11-next.yaml')

WARNING ⚠️ no model scale passed. Assuming scale='n'.


In [24]:
wandb.login(key="6e6412a1eff673cacdcda973f6e61422daaa9387")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/abk171/.netrc


NotImplementedError: ('{} cannot be pickled', '_MultiProcessingDataLoaderIter')

In [22]:


results = model.train(
    data='data_yolo.yaml',
    imgsz=[450, 1100],
    epochs=40,
    patience= 0,
    batch=16,
    workers=4,
    optimizer='AdamW',
    lr0=0.001, fliplr=0.5,crop_fraction=0.2,
    project="yolo",name="yolov11-next runs 40"
    )

Ultralytics 8.3.94 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (Quadro RTX 8000, 48407MiB)
engine/trainer: task=detect, mode=train, model=yolo11-next.yaml, data=data_yolo.yaml, epochs=40, time=None, patience=0, batch=16, imgsz=[450, 1100], save=True, save_period=-1, cache=False, device=None, workers=4, project=yolo, name=yolov11-next runs 40, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sho

train: Scanning /home/abk171/hexai_work/data_yolo/labels/train.cache... 101 images, 0 backgrounds, 0 c
val: Scanning /home/abk171/hexai_work/data_yolo/labels/val.cache... 19 images, 0 backgrounds, 0 corrup


Plotting labels to yolo/yolov11-next runs 40/labels.jpg... 
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 77 weight(decay=0.0), 98 weight(decay=0.0005), 89 bias(decay=0.0)
Image sizes 1120 train, 1120 val
Using 4 dataloader workers
Logging results to yolo/yolov11-next runs 40
Starting training for 40 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/40      7.31G      2.389       2.85      2.776         12       1120: 100%|██████████| 7/7 [0
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|███████


                   all         19         19     0.0833      0.211     0.0864     0.0232

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      40/40      7.46G      1.554      1.359      2.078          5       1120: 100%|██████████| 7/7 [0
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|███████

                   all         19         19      0.586      0.789      0.712       0.33



40 epochs completed in 0.021 hours.
Optimizer stripped from yolo/yolov11-next runs 40/weights/last.pt, 5.5MB
Optimizer stripped from yolo/yolov11-next runs 40/weights/best.pt, 5.5MB

Validating yolo/yolov11-next runs 40/weights/best.pt...
Ultralytics 8.3.94 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (Quadro RTX 8000, 48407MiB)
YOLO11-next summary (fused): 108 layers, 2,583,781 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|███████


                   all         19         19      0.774      0.721      0.781      0.389
Speed: 0.2ms preprocess, 1.1ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to yolo/yolov11-next runs 40


In [25]:
for i in badset:
    dicom_path = os.path.join(dicom_dir, i)
    img_array = load_dicom(dicom_path)
    img_jpg = dicom_to_jpg(img_array)

    img_filename = i.replace('.dcm', '.jpg')
    img_jpg.save(os.path.join(image_test_dir, img_filename))

In [26]:
results = model(image_test_dir, imgsz=[450,1100], save=True)


WARNING ⚠️ imgsz=[450, 1100] must be multiple of max stride 32, updating to [480, 1120]
image 1/3 /home/abk171/data_yolo/images/test/178.jpg: 480x1120 1 ulnar, 28.5ms
image 2/3 /home/abk171/data_yolo/images/test/95.jpg: 480x1120 (no detections), 8.0ms
image 3/3 /home/abk171/data_yolo/images/test/97.jpg: 480x1120 1 ulnar, 7.2ms
Speed: 10.9ms preprocess, 14.5ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 1120)
Results saved to runs/detect/train113
